In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
init_notebook_mode(connected=True)

In [2]:
cols = ['load_date', 'timestamp', 'date','exchange','symbol','price','volume','type','side']
df = pd.read_csv("data/history/Binance.ETH-USDT.csv", 
                 names=cols, 
                 parse_dates=['load_date', 'date'],
                 dtype={'exchange':'category',
                        'side':'category',
                        'type':'category',
                        'symbol':'category'
                       }
                ).set_index('date')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 237612 entries, 2018-05-17 12:36:35.443000 to 2018-05-19 22:05:07.716000
Data columns (total 8 columns):
load_date    237612 non-null datetime64[ns]
timestamp    237612 non-null int64
exchange     237612 non-null category
symbol       237612 non-null category
price        237612 non-null float64
volume       237612 non-null float64
type         237612 non-null category
side         237612 non-null category
dtypes: category(4), datetime64[ns](1), float64(2), int64(1)
memory usage: 10.0 MB


In [4]:
# localize
# df.index = df.index.tz_localize('UTC').tz_convert('Europe/Moscow')

In [5]:
df = df[['exchange','symbol','price','volume','side']]
df.tail(10)

,exchange,symbol,price,volume,side
date,,,,,
2018-05-19 22:05:03.319,Binance,ETH/USDT,694.16,0.00150,buy
2018-05-19 22:05:03.319,Binance,ETH/USDT,694.20,2.43519,buy
2018-05-19 22:05:03.377,Binance,ETH/USDT,694.76,7.47500,sell
2018-05-19 22:05:04.552,Binance,ETH/USDT,694.76,0.08831,sell
2018-05-19 22:05:04.253,Binance,ETH/USDT,694.21,0.04400,sell
2018-05-19 22:05:04.278,Binance,ETH/USDT,694.93,0.14406,buy
2018-05-19 22:05:05.320,Binance,ETH/USDT,694.80,0.00150,buy
2018-05-19 22:05:06.164,Binance,ETH/USDT,694.79,0.00476,buy
2018-05-19 22:05:07.716,Binance,ETH/USDT,694.79,0.58589,buy


In [6]:
df[['price','volume']].describe()

,price,volume
count,237612.000000,237612.000000
mean,686.326862,1.016170
std,13.141370,6.199131
min,656.000000,0.000010
25%,676.490000,0.020000
50%,686.330000,0.173990
75%,695.470000,0.660000
max,717.800000,1305.808330


In [7]:
#df.resample('60min', how={'price':'ohlc', 'volume':'sum'})
df_hours = df.resample('60min').agg({'price':'ohlc', 'volume':'sum'})

In [8]:
df_hours.tail()

price                              volume
                       open    high     low   close      volume
date                                                           
2018-05-19 18:00:00  701.32  704.33  700.20  701.78  1693.85678
2018-05-19 19:00:00  700.92  704.99  700.11  703.66  2284.38541
2018-05-19 20:00:00  704.00  704.81  697.10  697.40  1356.81126
2018-05-19 21:00:00  697.90  699.01  689.09  689.99  4461.95796
2018-05-19 22:00:00  689.28  694.97  688.89  694.80   606.38656

### Graph

In [9]:
data = [dict( type='candlestick',
              #x=df_hours.index.tz_localize('Europe/Moscow'), 
              x=df_hours.index + pd.DateOffset(hours=3),
              open=df_hours.price.open,
              high=df_hours.price.high,
              low=df_hours.price.low,
              close=df_hours.price.close,
              #volume=df_hours.volume.volume
              yaxis = 'y2',
              name = 'OHLC',
             )]

In [25]:
def movingaverage(interval, window_size=10):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

window_size = 5
mv_y = movingaverage(df_hours.price.close, window_size=window_size)
mv_x = list(df_hours.index)

# Clip the ends
mv_x = mv_x[window_size:-window_size]
mv_y = mv_y[window_size:-window_size]

In [26]:
def bbands(price, window_size=10, num_of_std=5):
    rolling_mean = price.rolling(window=window_size).mean()
    rolling_std  = price.rolling(window=window_size).std()
    upper_band = rolling_mean + (rolling_std*num_of_std)
    lower_band = rolling_mean - (rolling_std*num_of_std)
    return rolling_mean, upper_band, lower_band

bb_avg, bb_upper, bb_lower = bbands(df_hours.price.close)

In [27]:
layout = go.Layout(xaxis = dict(rangeslider = dict(visible = False)))
fig = go.Figure(data=data, layout=layout)

# OHLC
fig['data'] = data

# Volume
fig['data'].append( dict(x=df_hours.index + pd.DateOffset(hours=3), 
                         y=df_hours.volume.volume,           
                         type='bar',
                         yaxis='y', 
                         name='Volume',
                        ) )
# MA
fig['data'].append( dict( x=mv_x, y=mv_y, type='scatter', mode='lines', 
                         line = dict( width = 1 ),
                         marker = dict( color = '#E377C2' ),
                         yaxis = 'y2', name='Moving Average' ) )

# Bollinger bands
fig['data'].append( dict( x=df_hours.index, y=bb_upper, type='scatter', yaxis='y2', 
                         line = dict( width = 1 ),
                         marker=dict(color='#ccc'), hoverinfo='none', 
                         legendgroup='Bollinger Bands', name='Bollinger Bands') )

fig['data'].append( dict( x=df_hours.index, y=bb_lower, type='scatter', yaxis='y2',
                         line = dict( width = 1 ),
                         marker=dict(color='#ccc'), hoverinfo='none',
                         legendgroup='Bollinger Bands', showlegend=False ) )

fig['layout']['title'] = "{} {}".format(df.exchange[0].capitalize(), df.symbol[0])
fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
#fig['layout']['xaxis'] = dict( rangeselector = dict( visible = False ) )
#fig['layout']['xaxis'] = dict(showticklabels = True)
fig['layout']['yaxis'] = dict( domain = [0.0, 0.2], showticklabels = True )
fig['layout']['yaxis2'] = dict( domain = [0.2, 1.0] )
fig['layout']['legend'] = dict( orientation = 'h', y=0.97, x=0.3, yanchor='top' )
fig['layout']['margin'] = dict( t=40, b=50, r=40, l=40)

iplot(fig, filename='OHLC without Rangeslider', validate=False)